# Agent-to-Agent (A2A) Protocol with Azure OpenAI

This notebook demonstrates connecting to a locally running AgentCore agent (simulating AWS hosting) that uses Azure OpenAI as its backend.

## Prerequisites

1. Ensure `azure_agent.py` is running locally on port 8080.
2. The agent should be configured with Azure OpenAI credentials.


In [ ]:
// Load configuration settings
#!import config/Settings.cs 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId, embeddingEndpoint, embeddingApiKey) = Settings.LoadFromFile();

Console.WriteLine($"Configuration loaded:");
Console.WriteLine($"- Model: {model}");
Console.WriteLine($"- Endpoint: {azureEndpoint}");

In [ ]:
using System.Net.Http;
using System.Text;
using System.Text.Json;

// Define the agent endpoint
string agentUrl = "http://localhost:8080/";

// Helper function to send A2A message
async Task<string> SendMessageToAgent(string message, string conversationId)
{
    using var client = new HttpClient();
    
    // Construct the payload expected by BedrockAgentCoreApp
    // Based on sdk_agent.py: extract_prompt_from_payload expects "prompt", "session_id", "actor_id"
    var payload = new
    {
        prompt = message,
        session_id = conversationId,
        actor_id = "user"
    };

    var jsonPayload = JsonSerializer.Serialize(payload);
    var content = new StringContent(jsonPayload, Encoding.UTF8, "application/json");

    Console.WriteLine($"Sending message to {agentUrl}...");
    try 
    {
        // The agent might be using a specific path or just root. 
        // BedrockAgentCoreApp usually handles POST requests.
        var response = await client.PostAsync(agentUrl, content);
        response.EnsureSuccessStatusCode();
        
        var responseBody = await response.Content.ReadAsStringAsync();
        return responseBody;
    }
    catch (Exception ex)
    {
        return $"Error: {ex.Message}";
    }
}

Console.WriteLine("A2A Client helper created.");

In [ ]:
// Test the connection
string conversationId = Guid.NewGuid().ToString();
string query = "What time is it?";

Console.WriteLine($"Query: {query}");
string response = await SendMessageToAgent(query, conversationId);
Console.WriteLine($"Response:\n{response}");

In [ ]:
// Test with a tool call
query = "Echo this message: Hello from Semantic Kernel!";

Console.WriteLine($"Query: {query}");
response = await SendMessageToAgent(query, conversationId);
Console.WriteLine($"Response:\n{response}");